## Get all the 24 books at the page number 0



---





In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# URL of the main page containing all books
main_url = 'https://arab-books.com/page/0/'

# Function to get all book URLs from the main page using regex
def get_book_urls(main_url):
    book_urls = []
    response = requests.get(main_url)
    if response.status_code == 200:
        text = response.text
        url_pattern = r'https://arab-books\.com/books/[^\"]+/'
        url_regex = re.compile(url_pattern)
        matches = url_regex.findall(text)
        for match in matches:
          if match not in book_urls:
                book_urls.append(match)
    else:
        print("Failed to retrieve the main page.")
    return book_urls


In [2]:
books_urls = get_book_urls(main_url)
books_urls

['https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d8%af%d9%8a%d9%86-%d8%a7%d9%84%d9%82%d9%8a%d9%85-pdf/',
 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%aa%d8%b0%d9%83%d8%b1%d8%a9-%d9%8a%d8%a7-%d8%af%d8%b9%d8%a7%d8%a9-%d8%a7%d9%84%d8%a5%d8%b3%d9%84%d8%a7%d9%85-pdf/',
 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%aa%d8%a7%d8%b1%d9%8a%d8%ae-%d8%a7%d9%88%d8%b1%d8%b4%d9%84%d9%8a%d9%85-pdf/',
 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%86%d8%b8%d8%b1%d9%8a%d8%a9-%d8%a7%d9%84%d8%af%d9%88%d8%a7%d8%a6%d8%b1-%d8%a7%d9%84%d8%ab%d9%84%d8%a7%d8%ab-pdf/',
 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d9%85%d8%a7%d8%a1-%d9%88%d8%a7%d9%84%d8%a3%d8%ad%d9%84%d8%a7%d9%85-pdf/',
 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%ac%d8%af%d9%84%d9%8a%d8%a9-%d8%a7%d9%84%d8%b2%d9%85%d9%86-pdf/',
 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d9%86%d8%a7%d8%b1-pdf-%d8%a7%d9%84%d8%aa%d8%ad%d9%84

In [3]:
len(books_urls)

24


## Take the books in the first 4 pages

In [4]:
page_urls = [f'https://arab-books.com/page/{i}/' for i in range(5)]  # Extend the range as needed
# Collect URLs from all pages
all_book_urls = []
for url in page_urls:
    all_book_urls.extend(get_book_urls(url))

# Remove duplicates
unique_book_urls = list(set(all_book_urls))

print(f"Total unique URLs: {len(unique_book_urls)}")
print("Unique URLs:")
print(unique_book_urls)

Total unique URLs: 96
Unique URLs:
['https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d9%84%d9%88%d8%b3%d8%b7%d9%8a%d8%a9-%d9%81%d9%8a-%d8%b6%d9%88%d8%a1-%d8%a7%d9%84%d9%82%d8%b1%d8%a2%d9%86-%d8%a7%d9%84%d9%83%d8%b1%d9%8a%d9%85-pdf/', 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%ae%d8%aa%d9%85-%d8%a7%d9%84%d9%86%d8%a8%d9%88%d8%a9-pdf/', 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%a7%d8%b3%d8%aa%d8%ae%d8%af%d8%a7%d9%85-%d9%86%d8%b8%d9%85-%d8%a7%d9%84%d9%85%d8%b9%d9%84%d9%88%d9%85%d8%a7%d8%aa-%d8%a7%d9%84%d8%ac%d8%ba%d8%b1%d8%a7%d9%81%d9%8a%d8%a9/', 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d9%86%d8%b2%d9%84-%d8%a7%d9%84%d9%85%d8%b3%d8%a7%d9%83%d9%8a%d9%86-pdf/', 'https://arab-books.com/books/%d9%83%d8%aa%d8%a7%d8%a8-%d8%ae%d9%88%d8%a7%d8%b7%d8%b1%d9%8a-%d8%ad%d9%88%d9%84-%d9%85%d8%a7-%d9%82%d9%8a%d9%84-%d8%b9%d9%86-%d8%b4%d9%8a%d8%a6%d9%8a%d8%a9-%d8%a7%d9%84%d9%85%d8%b9%d8%af%d9%88-2/', 'https://arab-books.com/books/%d9%83%d8%aa%d8%

# Web Scrapping : Extract the important information about the book

In [5]:
arabic_text_pattern = re.compile(r'[ء-ي\s]+')

# Function to extract book information from a given book URL
def extract_book_info(url):
    book_info = {}
    combined_text = ""  # Initialize combined_text variable
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'lxml')
            # Extract information from 'li' tags with class 'mb-2'
            for li in soup.find_all('li', class_='mb-2'):
                key_tag = li.find('span', class_='fw-bold')
                if key_tag:
                    key = key_tag.text.strip(':')
                    value_tag = li.find('a')
                    if value_tag:
                        value = ', '.join([a.text for a in li.find_all('a')])
                    else:
                        value = li.find('span', class_='fw-bold').next_sibling.strip()
                    book_info[key] = value

            # for div in soup.find_all('div', class_='col-auto'):
            # a_tag = soup.find_all('a', id='book-read')
            # if a_tag:
            #     link = a_tag['href']
            #     book_info['link'] = link


            book_info['الملخص'] = combined_text.strip()
            # Extract and clean Arabic text from div elements
            for div in soup.find_all('div', class_='mb-3 border rounded p-md-3 p-2'):
                arabic_texts = arabic_text_pattern.findall(div.get_text())
                text = "".join(arabic_texts)
                result = re.sub(r'\s+', ' ', text)
                result = result.replace("الرئيسيةالكتبكتاب","")
                combined_text += result + "\n"

            book_info['الملخص'] = combined_text.strip()  # Add combined Arabic text to the dictionary
        else:
            print(f"Failed to retrieve the book page: {url}")
    except Exception as e:
        print(f"Error processing {url}: {e}")

    return book_info


In [6]:
all_books_info = []

for book_url in unique_book_urls:
  book_info = extract_book_info(book_url)
  if book_info:
      # Only add if there's valid info
    all_books_info.append(book_info)

all_books_info

[{'الإسم العربي': 'كتاب الوسطية في ضوء القرآن الكريم PDF',
  'المؤلف': 'الشيخ ناصر بن سليمان العمر',
  'الصفحات': '216',
  'الحجم': '1.23',
  'اللغة': 'العربية',
  'النوع': 'pdf',
  'الاقسام': 'كتب اسلامية',
  'الملخص': 'الوسطية في ضوء القرآن الكريم \n كتاب الوسطية في ضوء القرآن الكريم لا يوجد تقييمات لهذا الكتاب قيم الكتاب التقييم كتاب الوسطية في ضوء القرآن الكريم الغاء حفظ التقييم اضافة للمفضلة الإسم العربي كتاب الوسطية في ضوء القرآن الكريم المؤلف الشيخ ناصر بن سليمان العمر الصفحات الحجم اللغة العربية النوع الاقسام كتب اسلامية قراءة تحميل هذا الكتاب ملكية عامة نشر هذا الكتاب برخصة الملكية العامة او بموافقة المؤلف لك حقوق ملكية اتصال بنا \n كتاب الوسطية في ضوء القرآن الكريم تلخيص كتاب الوسطية في ضوء القرآن الكريم للشيخ ناصر بن سليمان العمر للشيخ ناصر بن سليمان العمر كتب مميزة حظيت بشعبية كبيرة وأثرت في العديد من القراء تتناول كتبه موضوعات دينية وفلسفية بشكل عميق وملهم كما أن لديه مجموعة من الكتب والمقالات التي تتنوع في مواضيعها وتجذب القراء من مختلف الفئات في هذا الكتاب تبرز أهمية الو

In [7]:
all_books_info[0]


{'الإسم العربي': 'كتاب الوسطية في ضوء القرآن الكريم PDF',
 'المؤلف': 'الشيخ ناصر بن سليمان العمر',
 'الصفحات': '216',
 'الحجم': '1.23',
 'اللغة': 'العربية',
 'النوع': 'pdf',
 'الاقسام': 'كتب اسلامية',
 'الملخص': 'الوسطية في ضوء القرآن الكريم \n كتاب الوسطية في ضوء القرآن الكريم لا يوجد تقييمات لهذا الكتاب قيم الكتاب التقييم كتاب الوسطية في ضوء القرآن الكريم الغاء حفظ التقييم اضافة للمفضلة الإسم العربي كتاب الوسطية في ضوء القرآن الكريم المؤلف الشيخ ناصر بن سليمان العمر الصفحات الحجم اللغة العربية النوع الاقسام كتب اسلامية قراءة تحميل هذا الكتاب ملكية عامة نشر هذا الكتاب برخصة الملكية العامة او بموافقة المؤلف لك حقوق ملكية اتصال بنا \n كتاب الوسطية في ضوء القرآن الكريم تلخيص كتاب الوسطية في ضوء القرآن الكريم للشيخ ناصر بن سليمان العمر للشيخ ناصر بن سليمان العمر كتب مميزة حظيت بشعبية كبيرة وأثرت في العديد من القراء تتناول كتبه موضوعات دينية وفلسفية بشكل عميق وملهم كما أن لديه مجموعة من الكتب والمقالات التي تتنوع في مواضيعها وتجذب القراء من مختلف الفئات في هذا الكتاب تبرز أهمية الوسطية كمن

In [8]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
df = pd.DataFrame(all_books_info)
df

,الإسم العربي,المؤلف,الصفحات,الحجم,اللغة,النوع,الاقسام,الملخص
0,كتاب الوسطية في ضوء القرآن الكريم PDF,الشيخ ناصر بن سليمان العمر,216,1.23,العربية,pdf,كتب اسلامية,الوسطية في ضوء القرآن الكريم \n كتاب الوسطية ف...
1,NaN,الكاتب أبو الأعلى المودودي,58,1.74,NaN,pdf,كتب اسلامية,ختم النبوة \n كتاب ختم النبوة لا يوجد تقييمات ...
2,NaN,عبد الله بن حسين القاضي,15,0.88,NaN,pdf,كتب الأدب العربي,استخدام نظم المعلومات الجغرافية في التنمية الا...
3,كتاب نزل المساكين,الطاهر بن جلون,252,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",نزل المساكين \n كتاب نزل المساكين لا يوجد تقيي...
4,كتاب خواطري حول ما قيل عن شيئية المعدوم عند ال...,دكتور محمود محمد علي,23,3,العربية,pdf,كتب الفلسفة المعاصرة,خواطري حول ما قيل عن شيئية المعدوم عند المعتزل...
...,...,...,...,...,...,...,...,...
91,كتاب النحو اليوناني وسبل إنتقاله إلى السريان Pdf,دكتور محمود محمد علي,140,16,العربية,pdf,كتب الفلسفة المعاصرة,النحو اليوناني وسبل إنتقاله إلى السريان \n كتا...
92,طاهر أبو فاشا قيثارة الإذاعة وراهب الليل,دكتور محمود محمد علي,23,1,العربية,pdf,NaN,طاهر أبو فاشا قيثارة الإذاعة وراهب الليل \n كت...
93,NaN,عبد الله بن حسين القاضي,86,12.16,NaN,pdf,كتب الأدب العربي,خرائط طريق الهجرة النبوية \n كتاب خرائط طريق ا...
94,كتاب المشير حسين طنطاوي أيقونة الوطنية ورمز ال...,NaN,25,1,العربية,pdf,كتب الفلسفة المعاصرة,المشير حسين طنطاوي أيقونة الوطنية ورمز الكرامة...


## Apply clustering on the book categories




In [11]:
df['الاقسام'].isnull().sum()


4

In [12]:
df.dropna(subset=['الاقسام'], inplace=True) # drop rows with NaN in the 'الاقسام' column


In [13]:
df['الاقسام'].isnull().sum()


0

In [ ]:
df['الاقسام'].unique()

array(['كتب الفلسفة المعاصرة', 'كتب الأدب العربي, كتب روايات عربية',
       'قضايا', 'كتب اسلامية, كتب الأدب العربي', 'كتب اسلامية',
       'كتب الأدب العربي', 'كتب الادب, كتب الفلسفة المعاصرة',
       'كتب علوم القرآن', 'كتب تفسير القرآن الكريم', 'كتب تفاسير',
       'كتب أصول الفقه', 'كتب متنوعة',
       'كتب الادب, كتب العقيدة, كتب الفلسفة المعاصرة'], dtype=object)

In [ ]:
cateogories=df['الاقسام'].unique()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cateogories)

print(X.toarray())  # To see the TF-IDF vectors


[[0.         0.         0.         0.         0.         0.
  0.         0.67005014 0.         0.         0.67005014 0.
  0.         0.         0.         0.         0.         0.31947711
  0.        ]
 [0.         0.         0.39748219 0.         0.39748219 0.
  0.         0.         0.         0.         0.         0.
  0.         0.51978253 0.51978253 0.         0.         0.3790357
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         1.         0.
  0.        ]
 [0.         0.55150785 0.48905465 0.         0.48905465 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.46635843
  0.        ]
 [0.         0.92105774 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.38942604
  0.        ]

In [ ]:
from sklearn.cluster import KMeans

# Number of clusters
num_clusters = 9

# K-Means Clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(X)

# Add the cluster labels to the DataFrame
df_clustering = pd.DataFrame({'Category': cateogories, 'Cluster': kmeans.labels_})



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
df_clustering

,Category,Cluster
0,كتب الفلسفة المعاصرة,1
1,"كتب الأدب العربي, كتب روايات عربية",3
2,قضايا,7
3,"كتب اسلامية, كتب الأدب العربي",3
4,كتب اسلامية,2
5,كتب الأدب العربي,3
6,"كتب الادب, كتب الفلسفة المعاصرة",1
7,كتب علوم القرآن,0
8,كتب تفسير القرآن الكريم,8
9,كتب تفاسير,6


In [ ]:
uster_groups = df_clustering.groupby('Cluster')['Category'].apply(list) # Use df_clustering instead of df

# Print each cluster and its members
for cluster, books in cluster_groups.items():
    print(f"Cluster {cluster}:")
    for book in books:
        print(f" - {book}")
    print("\n")

Cluster 0:
 - كتب علوم القرآن


Cluster 1:
 - كتب الفلسفة المعاصرة
 - كتب الادب, كتب الفلسفة المعاصرة
 - كتب الادب, كتب العقيدة, كتب الفلسفة المعاصرة


Cluster 2:
 - كتب اسلامية


Cluster 3:
 - كتب الأدب العربي, كتب روايات عربية
 - كتب اسلامية, كتب الأدب العربي
 - كتب الأدب العربي


Cluster 4:
 - كتب متنوعة


Cluster 5:
 - كتب أصول الفقه


Cluster 6:
 - كتب تفاسير


Cluster 7:
 - قضايا


Cluster 8:
 - كتب تفسير القرآن الكريم




# Text Preprocessing

In [14]:
df['الإسم العربي'].isnull().sum()

21

In [15]:
df['المؤلف'].isnull().sum()

4

In [16]:
df.dropna(subset=['المؤلف'], inplace=True) # drop rows with NaN in the 'الاقسام' column


In [17]:
df['المؤلف'].isnull().sum()

0

In [18]:
!pip install pyarabic

## Handle diacritics (تشكيل الكلمه)

In [19]:
import pyarabic.araby as araby

def handle_diacritics(text, method='remove'):
    if method == 'remove':
        return araby.strip_diacritics(text)


## Normalize some arabic letters

In [20]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("ڤ", "ف", text)
    text = re.sub("چ", "ج", text)
    text = re.sub("پ", "ب", text)
    text = re.sub("ڜ", "ش", text)
    text = re.sub("ڪ", "ك", text)
    text = re.sub("ٱ", "ا", text)
    return text

In [21]:
df['summary'] = df['الملخص']

In [22]:
df['summary']= df['summary'].apply(normalize_arabic)
df['summary'] = df['summary'].apply(handle_diacritics)
df

,الإسم العربي,المؤلف,الصفحات,الحجم,اللغة,النوع,الاقسام,الملخص,summary
0,كتاب الوسطية في ضوء القرآن الكريم PDF,الشيخ ناصر بن سليمان العمر,216,1.23,العربية,pdf,كتب اسلامية,الوسطية في ضوء القرآن الكريم \n كتاب الوسطية ف...,الوسطيه في ضوء القران الكريم \n كتاب الوسطيه ف...
1,NaN,الكاتب أبو الأعلى المودودي,58,1.74,NaN,pdf,كتب اسلامية,ختم النبوة \n كتاب ختم النبوة لا يوجد تقييمات ...,ختم النبوه \n كتاب ختم النبوه لا يوجد تقييمات ...
2,NaN,عبد الله بن حسين القاضي,15,0.88,NaN,pdf,كتب الأدب العربي,استخدام نظم المعلومات الجغرافية في التنمية الا...,استخدام نظم المعلومات الجغرافيه في التنميه الا...
3,كتاب نزل المساكين,الطاهر بن جلون,252,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",نزل المساكين \n كتاب نزل المساكين لا يوجد تقيي...,نزل المساكين \n كتاب نزل المساكين لا يوجد تقيي...
4,كتاب خواطري حول ما قيل عن شيئية المعدوم عند ال...,دكتور محمود محمد علي,23,3,العربية,pdf,كتب الفلسفة المعاصرة,خواطري حول ما قيل عن شيئية المعدوم عند المعتزل...,خواطري حول ما قيل عن شيئيه المعدوم عند المعتزل...
...,...,...,...,...,...,...,...,...,...
89,كتاب ليدبروا آياته PDF,الشيخ ناصر بن سليمان العمر,352,10.18,العربية,pdf,كتب اسلامية,ليدبروا آياته \n كتاب ليدبروا آياته لا يوجد تق...,ليدبروا اياته \n كتاب ليدبروا اياته لا يوجد تق...
90,كتاب الأشجار واغتيال مرزوق,عبد الرحمن منيف,190,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",الأشجار واغتيال مرزوق \n كتاب الأشجار واغتيال ...,الاشجار واغتيال مرزوق \n كتاب الاشجار واغتيال ...
91,كتاب النحو اليوناني وسبل إنتقاله إلى السريان Pdf,دكتور محمود محمد علي,140,16,العربية,pdf,كتب الفلسفة المعاصرة,النحو اليوناني وسبل إنتقاله إلى السريان \n كتا...,النحو اليوناني وسبل انتقاله الي السريان \n كتا...
93,NaN,عبد الله بن حسين القاضي,86,12.16,NaN,pdf,كتب الأدب العربي,خرائط طريق الهجرة النبوية \n كتاب خرائط طريق ا...,خرائط طريق الهجره النبويه \n كتاب خرائط طريق ا...


In [23]:
# Now you can join the strings in the column
text = ' '.join(df['summary'])
text=  re.sub(r'[^\w\s]','', text)  # Remove punctuation
text = re.sub(r'[^أ-ى\s]','', text)  # Remove symbols and numbers not in Arabic Unicode range
df

,الإسم العربي,المؤلف,الصفحات,الحجم,اللغة,النوع,الاقسام,الملخص,summary
0,كتاب الوسطية في ضوء القرآن الكريم PDF,الشيخ ناصر بن سليمان العمر,216,1.23,العربية,pdf,كتب اسلامية,الوسطية في ضوء القرآن الكريم \n كتاب الوسطية ف...,الوسطيه في ضوء القران الكريم \n كتاب الوسطيه ف...
1,NaN,الكاتب أبو الأعلى المودودي,58,1.74,NaN,pdf,كتب اسلامية,ختم النبوة \n كتاب ختم النبوة لا يوجد تقييمات ...,ختم النبوه \n كتاب ختم النبوه لا يوجد تقييمات ...
2,NaN,عبد الله بن حسين القاضي,15,0.88,NaN,pdf,كتب الأدب العربي,استخدام نظم المعلومات الجغرافية في التنمية الا...,استخدام نظم المعلومات الجغرافيه في التنميه الا...
3,كتاب نزل المساكين,الطاهر بن جلون,252,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",نزل المساكين \n كتاب نزل المساكين لا يوجد تقيي...,نزل المساكين \n كتاب نزل المساكين لا يوجد تقيي...
4,كتاب خواطري حول ما قيل عن شيئية المعدوم عند ال...,دكتور محمود محمد علي,23,3,العربية,pdf,كتب الفلسفة المعاصرة,خواطري حول ما قيل عن شيئية المعدوم عند المعتزل...,خواطري حول ما قيل عن شيئيه المعدوم عند المعتزل...
...,...,...,...,...,...,...,...,...,...
89,كتاب ليدبروا آياته PDF,الشيخ ناصر بن سليمان العمر,352,10.18,العربية,pdf,كتب اسلامية,ليدبروا آياته \n كتاب ليدبروا آياته لا يوجد تق...,ليدبروا اياته \n كتاب ليدبروا اياته لا يوجد تق...
90,كتاب الأشجار واغتيال مرزوق,عبد الرحمن منيف,190,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",الأشجار واغتيال مرزوق \n كتاب الأشجار واغتيال ...,الاشجار واغتيال مرزوق \n كتاب الاشجار واغتيال ...
91,كتاب النحو اليوناني وسبل إنتقاله إلى السريان Pdf,دكتور محمود محمد علي,140,16,العربية,pdf,كتب الفلسفة المعاصرة,النحو اليوناني وسبل إنتقاله إلى السريان \n كتا...,النحو اليوناني وسبل انتقاله الي السريان \n كتا...
93,NaN,عبد الله بن حسين القاضي,86,12.16,NaN,pdf,كتب الأدب العربي,خرائط طريق الهجرة النبوية \n كتاب خرائط طريق ا...,خرائط طريق الهجره النبويه \n كتاب خرائط طريق ا...


## Split the words into tokens

In [33]:
# df['summary'] =  df['summary'].apply(word_tokenize)


In [24]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Check the spelling of words in arabic then perform auto correction

In [27]:
!pip install ar_corrector

In [28]:
from ar_corrector.corrector import Corrector
import pandas as pd
from nltk.stem import ISRIStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize Corrector, Stemmer, and Lemmatizer
corr = Corrector()
stemmer = ISRIStemmer()
lemmatizer = WordNetLemmatizer()

# Load Arabic stop words
stop_words = set(stopwords.words('arabic'))

# Correct text in the DataFrame
for index, summary in df['summary'].items():
    if isinstance(summary, list):
        corrected_summary = corr.contextual_correct(' '.join(summary))
    elif isinstance(summary, str):
        corrected_summary = corr.contextual_correct(summary)
    else:
        corrected_summary = summary
    # Tokenize the corrected summary
    words = word_tokenize(corrected_summary)
    # Remove stop words
    filtered_words = [word for word in words if word not in stop_words]

    # Perform stemming
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    # Perform lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]

    # Update DataFrame with lemmatized text
    df.at[index, 'summary'] = lemmatized_words

# Check results
df

,الإسم العربي,المؤلف,الصفحات,الحجم,اللغة,النوع,الاقسام,الملخص,summary
0,كتاب الوسطية في ضوء القرآن الكريم PDF,الشيخ ناصر بن سليمان العمر,216,1.23,العربية,pdf,كتب اسلامية,الوسطية في ضوء القرآن الكريم \n كتاب الوسطية ف...,"[وسط, ضوء, قرن, كرم, كتب, وسط, ضوء, قرن, كرم, ..."
1,NaN,الكاتب أبو الأعلى المودودي,58,1.74,NaN,pdf,كتب اسلامية,ختم النبوة \n كتاب ختم النبوة لا يوجد تقييمات ...,"[ختم, نبه, كتب, ختم, نبه, وجد, قيم, لهذا, كتب,..."
2,NaN,عبد الله بن حسين القاضي,15,0.88,NaN,pdf,كتب الأدب العربي,استخدام نظم المعلومات الجغرافية في التنمية الا...,"[خدم, نظم, علم, جغراف, نمه, اجتماعيه, نطق, حضر..."
3,كتاب نزل المساكين,الطاهر بن جلون,252,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",نزل المساكين \n كتاب نزل المساكين لا يوجد تقيي...,"[نزل, ساك, كتب, نزل, ساك, وجد, قيم, لهذا, كتب,..."
4,كتاب خواطري حول ما قيل عن شيئية المعدوم عند ال...,دكتور محمود محمد علي,23,3,العربية,pdf,كتب الفلسفة المعاصرة,خواطري حول ما قيل عن شيئية المعدوم عند المعتزل...,"[خطر, حول, قيل, شخص, عدم, عزل, كتب, خطر, حول, ..."
...,...,...,...,...,...,...,...,...,...
89,كتاب ليدبروا آياته PDF,الشيخ ناصر بن سليمان العمر,352,10.18,العربية,pdf,كتب اسلامية,ليدبروا آياته \n كتاب ليدبروا آياته لا يوجد تق...,"[عبر, يته, كتب, عبر, يته, وجد, قيم, لهذا, كتب,..."
90,كتاب الأشجار واغتيال مرزوق,عبد الرحمن منيف,190,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",الأشجار واغتيال مرزوق \n كتاب الأشجار واغتيال ...,"[شجر, غيل, رزق, كتب, شجر, غيل, رزق, وجد, قيم, ..."
91,كتاب النحو اليوناني وسبل إنتقاله إلى السريان Pdf,دكتور محمود محمد علي,140,16,العربية,pdf,كتب الفلسفة المعاصرة,النحو اليوناني وسبل إنتقاله إلى السريان \n كتا...,"[نحو, يون, سبل, نقل, الي, سري, كتب, نحو, يون, ..."
93,NaN,عبد الله بن حسين القاضي,86,12.16,NaN,pdf,كتب الأدب العربي,خرائط طريق الهجرة النبوية \n كتاب خرائط طريق ا...,"[خرط, طرق, هجر, نبي, كتب, خرط, طرق, هجر, نبي, ..."


In [29]:
# Save the DataFrame to a CSV file
df.to_csv('my_Filtereddata.csv', index=False, encoding='utf-8-sig')  # Use 'utf-8-sig' for better compatibility with Excel


In [30]:
df['Original_Summary'] = df['الملخص']  # Keep the original summaries


## Search using the summary of the book

In [31]:
# new TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
corpus= df['summary'].apply(lambda x: ' '.join(x)) # Join the list of words into a single string for each document
cv_tfidf = TfidfVectorizer()
X_tfidf = cv_tfidf.fit_transform(corpus).toarray()
pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names_out()) # Use get_feature_names_out() instead of get_feature_names()

,ءدم,أثر,أجيال,أحكام,أحوال,أسعار,أصدقائ,أعط,أكد,ألف,...,يهم,يهه,يواجهونه,يوج,يوح,يود,يور,يوغوسلاف,يوم,يون
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.026942,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.064540,0.000000
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.017677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,0.046591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.017012,0.000000
84,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.042771,0.014101,0.000000
85,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.254680
86,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034652,0.000000,...,0.0,0.0,0.0,0.032381,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [67]:
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations # Import the combinations function from itertools

def search_documents(query, top_n=5):
    # Vectorize the query
    query_vector = cv_tfidf.transform([query]).toarray()

    # Compute cosine similarity between the query and the documents
    similarities = cosine_similarity(query_vector, X_tfidf).flatten()

    # Get the indices of the top_n most similar documents
    top_indices = similarities.argsort()[-top_n:][::-1]

    # Retrieve and print the details of the top_n similar documents
    results = []

    for index in top_indices:
      if similarities[index] > 0.1:
        result = {
            'Similarity': similarities[index],
            'Book Name': df['الإسم العربي'].iloc[index],
            'Author': df['المؤلف'].iloc[index],
            'Cateogory': df['الاقسام'].iloc[index],
            'Summary': df['Original_Summary'].iloc[index]
        }
        results.append(result)

    return results

# Example search
query="دكتور محمود محمد علي"
top_documents = search_documents(query, top_n=5)

# Print the search results
for result in top_documents:
    print(f"Similarity: {result['Similarity']}")
    print(f"Book Name: {result['Book Name']}")
    print(f"Author: {result['Author']}")
    print(f"Category: {result['Cateogory']}")
    print(f"Summary: {result['Summary']}")
    print("\n")

Similarity: 0.25532247662488916
Book Name: كتاب قراءة في كتاب فلسفة هيوم الأخلاقية للأستاذ الدكتور محمد محمد مدين Pdf
Author: دكتور محمود محمد علي
Category: كتب الفلسفة المعاصرة
Summary: قراءة في كتاب فلسفة هيوم الأخلاقية للأستاذ الدكتور محمد محمد مدين 
 كتاب قراءة في كتاب فلسفة هيوم الأخلاقية للأستاذ الدكتور محمد محمد مدين لا يوجد تقييمات لهذا الكتاب قيم الكتاب التقييم كتاب قراءة في كتاب فلسفة هيوم الأخلاقية للأستاذ الدكتور محمد محمد مدين الغاء حفظ التقييم اضافة للمفضلة الإسم العربي كتاب قراءة في كتاب فلسفة هيوم الأخلاقية للأستاذ الدكتور محمد محمد مدين المؤلف دكتور محمود محمد علي الصفحات الحجم اللغة العربية النوع الاقسام كتب الفلسفة المعاصرة قراءة تحميل هذا الكتاب ملكية عامة نشر هذا الكتاب برخصة الملكية العامة او بموافقة المؤلف لك حقوق ملكية اتصال بنا 
 كتاب قراءة في كتاب فلسفة هيوم الأخلاقية للأستاذ الدكتور محمد محمد مدين تلخيص كتاب قراءة في كتاب فلسفة هيوم الأخلاقية للأستاذ الدكتور محمد محمد مدين يعد كتاب قراءة في كتاب فلسفة هيوم الأخلاقية للأستاذ الدكتور محمد محمد مدين هو أشهر الكت

#Preproceessing to categories column

In [33]:
df['cateogories']=df['الاقسام']

In [34]:
df['cateogories'].apply(normalize_arabic)
df['cateogories'] = df['cateogories'].apply(handle_diacritics)
df

,الإسم العربي,المؤلف,الصفحات,الحجم,اللغة,النوع,الاقسام,الملخص,summary,Original_Summary,cateogories
0,كتاب الوسطية في ضوء القرآن الكريم PDF,الشيخ ناصر بن سليمان العمر,216,1.23,العربية,pdf,كتب اسلامية,الوسطية في ضوء القرآن الكريم \n كتاب الوسطية ف...,"[وسط, ضوء, قرن, كرم, كتب, وسط, ضوء, قرن, كرم, ...",الوسطية في ضوء القرآن الكريم \n كتاب الوسطية ف...,كتب اسلامية
1,NaN,الكاتب أبو الأعلى المودودي,58,1.74,NaN,pdf,كتب اسلامية,ختم النبوة \n كتاب ختم النبوة لا يوجد تقييمات ...,"[ختم, نبه, كتب, ختم, نبه, وجد, قيم, لهذا, كتب,...",ختم النبوة \n كتاب ختم النبوة لا يوجد تقييمات ...,كتب اسلامية
2,NaN,عبد الله بن حسين القاضي,15,0.88,NaN,pdf,كتب الأدب العربي,استخدام نظم المعلومات الجغرافية في التنمية الا...,"[خدم, نظم, علم, جغراف, نمه, اجتماعيه, نطق, حضر...",استخدام نظم المعلومات الجغرافية في التنمية الا...,كتب الأدب العربي
3,كتاب نزل المساكين,الطاهر بن جلون,252,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",نزل المساكين \n كتاب نزل المساكين لا يوجد تقيي...,"[نزل, ساك, كتب, نزل, ساك, وجد, قيم, لهذا, كتب,...",نزل المساكين \n كتاب نزل المساكين لا يوجد تقيي...,"كتب الأدب العربي, كتب روايات عربية"
4,كتاب خواطري حول ما قيل عن شيئية المعدوم عند ال...,دكتور محمود محمد علي,23,3,العربية,pdf,كتب الفلسفة المعاصرة,خواطري حول ما قيل عن شيئية المعدوم عند المعتزل...,"[خطر, حول, قيل, شخص, عدم, عزل, كتب, خطر, حول, ...",خواطري حول ما قيل عن شيئية المعدوم عند المعتزل...,كتب الفلسفة المعاصرة
...,...,...,...,...,...,...,...,...,...,...,...
89,كتاب ليدبروا آياته PDF,الشيخ ناصر بن سليمان العمر,352,10.18,العربية,pdf,كتب اسلامية,ليدبروا آياته \n كتاب ليدبروا آياته لا يوجد تق...,"[عبر, يته, كتب, عبر, يته, وجد, قيم, لهذا, كتب,...",ليدبروا آياته \n كتاب ليدبروا آياته لا يوجد تق...,كتب اسلامية
90,كتاب الأشجار واغتيال مرزوق,عبد الرحمن منيف,190,7.08,العربية,pdf,"كتب الأدب العربي, كتب روايات عربية",الأشجار واغتيال مرزوق \n كتاب الأشجار واغتيال ...,"[شجر, غيل, رزق, كتب, شجر, غيل, رزق, وجد, قيم, ...",الأشجار واغتيال مرزوق \n كتاب الأشجار واغتيال ...,"كتب الأدب العربي, كتب روايات عربية"
91,كتاب النحو اليوناني وسبل إنتقاله إلى السريان Pdf,دكتور محمود محمد علي,140,16,العربية,pdf,كتب الفلسفة المعاصرة,النحو اليوناني وسبل إنتقاله إلى السريان \n كتا...,"[نحو, يون, سبل, نقل, الي, سري, كتب, نحو, يون, ...",النحو اليوناني وسبل إنتقاله إلى السريان \n كتا...,كتب الفلسفة المعاصرة
93,NaN,عبد الله بن حسين القاضي,86,12.16,NaN,pdf,كتب الأدب العربي,خرائط طريق الهجرة النبوية \n كتاب خرائط طريق ا...,"[خرط, طرق, هجر, نبي, كتب, خرط, طرق, هجر, نبي, ...",خرائط طريق الهجرة النبوية \n كتاب خرائط طريق ا...,كتب الأدب العربي


In [35]:
stemmer = ISRIStemmer()
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # Tokenize
    tokens = nltk.word_tokenize(text)

    # Lowercase and remove stopwords and punctuation
    tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]

    # # Stemming
    tokens = [stemmer.stem(word) for word in tokens]
    tokens = [lemmatizer.lemmatize(word) for word in stemmed_words]
    return ' '.join(tokens)


In [36]:
df['cateogories'] = df['cateogories'].apply(preprocess_text)

In [39]:
df['Combined_Text'] = df['cateogories'] +' '+ df['summary'].str.join(' ')


# Apply the TF-IDF Vectorizer


## search using the cateogory name

In [41]:
# new TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
merged_corpus= df['cateogories'] # Join the list of words into a single string for each document
cv_tfidf = TfidfVectorizer() # Use a single TfidfVectorizer object
X_tfidf_merged = cv_tfidf.fit_transform(merged_corpus).toarray()
pd.DataFrame(X_tfidf_merged, columns=cv_tfidf.get_feature_names_out()) # Use the fitted object to get feature names

,ؤثر,ؤكد,ؤلف,اجتماعيه,ادب,ادو,ارخ,ازل,اكد,الل,...,يتم,يخت,يسا,يشر,يعد,يقظ,يلي,يمك,يمكن,يوم
0,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253
1,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253
2,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253
3,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253
4,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253
84,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253
85,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253
86,0.01253,0.01253,0.025061,0.037591,0.100243,0.01253,0.062652,0.01253,0.01253,0.087713,...,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.01253,0.025061,0.01253


In [42]:
def search_documents(query, top_n=5):
    # Vectorize the query
    query_vector = cv_tfidf.transform([query]).toarray()

    # Compute cosine similarity between the query and the documents
    similarities = cosine_similarity(query_vector, X_tfidf_merged).flatten()

    # Get the indices of the top_n most similar documents
    top_indices = similarities.argsort()[-top_n:][::-1]

    # Retrieve and print the details of the top_n similar documents
    results = []

    for index in top_indices:
      if similarities[index] > 0.2:
        result = {
            'Similarity': similarities[index],
            'Book Name': df['الإسم العربي'].iloc[index],
            'Author': df['المؤلف'].iloc[index],
            'Cateogory': df['الاقسام'].iloc[index],
            'Summary': df['Original_Summary'].iloc[index]
        }
        results.append(result)

    return results

# Example search
query="كتب الأدب العربي"
top_documents = search_documents(query, top_n=5)

# Print the search results
for result in top_documents:
    print(f"Similarity: {result['Similarity']}")
    print(f"Book Name: {result['Book Name']}")
    print(f"Author: {result['Author']}")
    print(f"Category: {result['Cateogory']}")
    print(f"Summary: {result['Summary']}")
    print("\n")

Similarity: 0.5513368898854234
Book Name: nan
Author: عبد الله بن حسين القاضي
Category: كتب اسلامية, كتب الأدب العربي
Summary: مفهوم الاتجاهات الجغرافية في القرآن الكريم الجزء الثاني
 كتاب مفهوم الاتجاهات الجغرافية في القرآن الكريم الجزء الثاني لا يوجد تقييمات لهذا الكتاب قيم الكتاب التقييم كتاب مفهوم الاتجاهات الجغرافية في القرآن الكريم الجزء الثاني الغاء حفظ التقييم اضافة للمفضلة المؤلف عبد الله بن حسين القاضي الصفحات الحجم النوع الاقسام كتب اسلامية كتب الأدب العربي قراءة تحميل هذا الكتاب ملكية عامة نشر هذا الكتاب برخصة الملكية العامة او بموافقة المؤلف لك حقوق ملكية اتصال بنا 
 كتاب مفهوم الاتجاهات الجغرافية في القرآن الكريم الجزء الثاني تحميل كتب عربي الروايات والكتب العربية تعتبر من الروابط بيننا وبين تاريخنا كعرب او بيننا وبين اللغة العربية وبين تاريخنا واحداث هذا التاريخ الروايات العربية الي الان ومع تقدم العصور والتكنولوجيا لازالت قائمة وحاضرة تلخيص كتاب مفهوم الاتجاهات الجغرافية في القرآن الكريمى للكاتب عبد الله بن حسين القاضي كتاب مفهوم الاتجاهات الجغرافية في القرآن الكريم للك

## Trying to combine the summary and the cateogory

In [43]:
# new TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
merged_corpus_2= df['Combined_Text'] # Join the list of words into a single string for each document
cv_tfidf = TfidfVectorizer() # Use a single TfidfVectorizer object
X_tfidf_merged_2 = cv_tfidf.fit_transform(merged_corpus_2).toarray()
pd.DataFrame(X_tfidf_merged_2, columns=cv_tfidf.get_feature_names_out()) # Us

,ءدم,أثر,أجيال,أحكام,أحوال,أسعار,أصدقائ,أعط,أكد,ألف,...,يهم,يهه,يواجهونه,يوج,يوح,يود,يور,يوغوسلاف,يوم,يون
0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.006348,0.000000
1,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.019453,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.033267,0.000000
2,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.005497,0.000000
3,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.006324,0.000000
4,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.006116,0.013881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,0.0325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.013555,0.000000
84,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.032847,0.012370,0.000000
85,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.006364,0.187776
86,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027683,0.000000,...,0.0,0.0,0.0,0.025869,0.0,0.0,0.0,0.000000,0.006306,0.000000


In [57]:
def search_documents(query, top_n=5):
    # Vectorize the query
    query_vector = cv_tfidf.transform([query]).toarray()

    # Compute cosine similarity between the query and the documents
    similarities = cosine_similarity(query_vector, X_tfidf_merged_2).flatten()

    # Get the indices of the top_n most similar documents
    top_indices = similarities.argsort()[-top_n:][::-1]

    # Retrieve and print the details of the top_n similar documents
    results = []

    for index in top_indices:
      if similarities[index] > 0.1:
        result = {
            'Similarity': similarities[index],
            'Book Name': df['الإسم العربي'].iloc[index],
            'Author': df['المؤلف'].iloc[index],
            'Cateogory': df['الاقسام'].iloc[index],
            'Summary': df['Original_Summary'].iloc[index]
        }
        results.append(result)

    return results

# Example search
query = """
دراسة للتحول من اتجاه الشرق إلى الشمال
كتاب دراسة للتحول من اتجاه الشرق إلى الشمال لا يوجد تقييمات لهذا الكتاب قيم الكتاب التقييم كتاب دراسة للتحول من اتجاه الشرق إلى الشمال الغاء حفظ التقييم اضافة للمفضلة المؤلف عبد الله بن حسين القاضي الصفحات الحجم النوع الاقسام كتب الأدب العربي قراءة تحميل هذا الكتاب ملكية عامة نشر هذا الكتاب برخصة الملكية العامة او بموافقة المؤلف لك حقوق ملكية اتصال بنا
 كتاب دراسة للتحول من اتجاه الشرق إلى الشمال تحميل كتب عربي الروايات والكتب العربية تعتبر من الروابط بيننا وبين تاريخنا كعرب او بيننا وبين اللغة العربية وبين تاريخنا واحداث هذا التاريخ الروايات العربية الي الان ومع تقدم العصور والتكنولوجيا لازالت قائمة وحاضرة تلخيص كتاب دراسة للتحول من اتجاه الشرق إلى الشمال للكاتب عبد الله بن حسين القاضي دراسة للتحول من اتجاه الشرق إلى الشمال للكاتب عبد الله بن حسين القاضي هو كتاب يتناول دراسة جغرافية وثقافية للتحولات التي تحدث في المجتمعات عند تغيير اتجاهاتها من الشرق إلى الشمال إليك ملخصا لأبرز ما جاء في الكتاب التمهيد والخلفية التاريخية يبدأ كتاب دراسة للتحول من اتجاه الشرق إلى الشمال بمقدمة توضح الأهمية التاريخية والجغرافية للاتجاهات الأربعة الشمال الجنوب الشرق الغرب وكيف كانت هذه الاتجاهات تؤثر على الثقافات والحضارات المختلفة يشرح الكاتب كيف كان الاتجاه نحو الشرق غالبا ما يرتبط بالشمس وطلوعها مما يرمز للبداية والنشاط بينما الشمال كان يعتبر اتجاها للبرودة والغموض التحولات الجغرافية والثقافية يركز الكاتب على التحولات الكبيرة التي شهدتها بعض المجتمعات عند تغيير مركز اهتمامها من الشرق إلى الشمال يتم مناقشة الأسباب الاقتصادية والسياسية التي قد تؤدي إلى مثل هذه التحولات مثل التغيرات في الطرق التجارية واكتشاف الموارد الطبيعية والتحولات في المناخ دراسات حالة يقدم الكاتب أمثلة محددة من التاريخ حيث قامت حضارات معينة بتحويل اهتمامها الجغرافي من الشرق إلى الشمال يناقش تأثير هذه التحولات على الثقافة الاقتصاد والسياسة في تلك المجتمعات التأثيرات النفسية والاجتماعية يتم التطرق إلى كيفية تأثير هذه التحولات على الوعي الجمعي للأفراد في المجتمع يشير الكاتب إلى تغييرات في الفنون الأدب والعمارة كنتيجة لهذه التحولات الاستنتاجات يختم كتاب دراسة للتحول من اتجاه الشرق إلى الشمال باستنتاجات حول أهمية فهم هذه التحولات الجغرافية والثقافية يقدم توصيات حول كيفية دراسة وتحليل التحولات المستقبلية بين المجتمعات المختلفة كتاب دراسة للتحول من اتجاه الشرق إلى الشمال يقدم رؤية شاملة ومتكاملة لفهم الديناميكيات الجغرافية والثقافية التي تؤثر على المجتمعات ويساعد في تسليط الضوء على الأسباب والنتائج المترتبة على التحولات من اتجاه إلى آخر نبذة عن كاتب كتاب دراسة للتحول من اتجاه الشرق إلى الشمال عبد الله بن حسين القاضي هو كاتب سعودي له إسهامات ملحوظة في الأدب والثقافة يبرز القاضي بكتاباته التي تتناول قضايا اجتماعية وثقافية متنوعة ويشتهر بأسلوبه المميز الذي يجمع بين التحليل العميق والتعبير الأدبي الرصين من أبرز أعماله الأدبية مجموعة من الكتب والمقالات التي نشرت في مختلف الصحف والمجلات الثقافية تتناول كتاباته مواضيع مثل الهوية الثقافية التحديات الاجتماعية والتحولات التي تشهدها المجتمعات العربية كما يشارك القاضي في العديد من الندوات والمناسبات الأدبية حيث يساهم في إثراء الحوار الثقافي والتواصل الفكري يعرف عبد الله بن حسين القاضي بشغفه بالكتابة والقراءة مما جعله شخصية مؤثرة في الوسط الثقافي السعودي والعربي بشكل عام يمكنك أيضا قراءة وتحميل كتب عربي مثل رواية الحصاد للسحار
"""
top_documents = search_documents(query, top_n=5)

# Print the search results
for result in top_documents:
    print(f"Similarity: {result['Similarity']}")
    print(f"Book Name: {result['Book Name']}")
    print(f"Author: {result['Author']}")
    print(f"Category: {result['Cateogory']}")
    print(f"Summary: {result['Summary']}")
    print("\n")

Similarity: 0.11304867364552151
Book Name: nan
Author: عبد الله بن حسين القاضي
Category: كتب الأدب العربي
Summary: دراسة للتحول من اتجاه الشرق إلى الشمال 
 كتاب دراسة للتحول من اتجاه الشرق إلى الشمال لا يوجد تقييمات لهذا الكتاب قيم الكتاب التقييم كتاب دراسة للتحول من اتجاه الشرق إلى الشمال الغاء حفظ التقييم اضافة للمفضلة المؤلف عبد الله بن حسين القاضي الصفحات الحجم النوع الاقسام كتب الأدب العربي قراءة تحميل هذا الكتاب ملكية عامة نشر هذا الكتاب برخصة الملكية العامة او بموافقة المؤلف لك حقوق ملكية اتصال بنا 
 كتاب دراسة للتحول من اتجاه الشرق إلى الشمال تحميل كتب عربي الروايات والكتب العربية تعتبر من الروابط بيننا وبين تاريخنا كعرب او بيننا وبين اللغة العربية وبين تاريخنا واحداث هذا التاريخ الروايات العربية الي الان ومع تقدم العصور والتكنولوجيا لازالت قائمة وحاضرة تلخيص كتاب دراسة للتحول من اتجاه الشرق إلى الشمال للكاتب عبد الله بن حسين القاضي دراسة للتحول من اتجاه الشرق إلى الشمال للكاتب عبد الله بن حسين القاضي هو كتاب يتناول دراسة جغرافية وثقافية للتحولات التي تحدث في المجتمعات عند تغيي

In [69]:
# from sklearn.metrics.pairwise import cosine_similarity
# from itertools import combinations # Import the combinations function from itertools

# pairs = list(combinations(range(len(corpus)),2))
# combos = [(corpus[a_index], corpus[b_index]) for (a_index, b_index) in pairs]

# results_tfidf = [cosine_similarity(X_tfidf[a_index].reshape(1, -1), X_tfidf[b_index].reshape(1, -1)) # Reshape the 1D arrays to 2D arrays using reshape(1, -1)

# for (a_index, b_index) in pairs]
# sorted(zip(results_tfidf, combos), reverse=True)

